<a href="https://colab.research.google.com/github/KIeccio/DevOps-EAD/blob/main/Code_Elevate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# instalando as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .getOrCreate()

# **Assignment - Data Engineer.**

---




# Diário de Bordo

  A amostra de dados em anexo (info_transportes.csv), possui dados de um aplicativo de transporte privado, cujas colunas são:
  
  * DATA_INICIO (formato: "mm-dd-yyyy HH")
  * DATA_FIM (formato: "mm-dd-yyyy HH")
  * CATEGORIA
  * LOCAL_INICIO
  * LOCAL_FIM
  * PROPOSITO
  * DISTANCIA
  
  Uma equipe está elaborando um modelo para compreender como os clientes estão utilizando o aplicativo. Para isso, você precisa fornecer uma nova tabela “info_corridas_do_dia", com dados agrupados pela data de início do transporte utilizando a formatação “yyyy-MM-dd”, contendo as seguintes colunas:
  
  Nome da coluna 	Descrição

  DT_REFE: 	Data de referência.

  QT_CORR: 	Quantidade de corridas.

  QT_CORR_NEG: 	Quantidade de corridas com a categoria “Negócio”.

  QT_CORR_PESS: 	Quantidade de corridas com a categoria “Pessoal”.

  VL_MAX_DIST: 	Maior distância percorrida por uma corrida.

  VL_MIN_DIST: 	Menor distância percorrida por uma corrida.

  VL_AVG_DIST: 	Média das distâncias percorridas.

  QT_CORR_REUNI: 	Quantidade de corridas com o propósito de "Reunião".

  QT_CORR_NAO_REUNI: 	Quantidade de corridas com o propósito declarado e diferente de "Reunião".


In [ ]:
import pyspark.sql.functions as F

In [ ]:
spark
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [ ]:
# Lendo arquivo CSV
info_transportes = spark.read.csv("/content/info_transportes.csv", inferSchema=True, header=True, sep=";")

In [ ]:
# Formatando o dataframe e a data para o proximo passo
info_transportes = info_transportes.withColumn(
    "DATA_INICIO_FORMATADA",
    F.date_format(
        F.to_timestamp("DATA_INICIO", "MM-dd-yyyy HH:mm"),
        "yyyy-MM-dd"
        )
).withColumn(
    "DATA_FIM_FORMATADA",
    F.date_format(
        F.to_timestamp("DATA_FIM", "MM-dd-yyyy HH:mm"),
        "yyyy-MM-dd"
        )
)

In [ ]:
# Removendo as colunas DATA_INICIO e DATA_FIM
info_transportes = info_transportes.drop("DATA_INICIO", "DATA_FIM")

In [ ]:
display(
    info_transportes.show()
)

+---------+-----------------+---------------+---------+-----------------+---------------------+------------------+
|CATEGORIA|     LOCAL_INICIO|      LOCAL_FIM|DISTANCIA|        PROPOSITO|DATA_INICIO_FORMATADA|DATA_FIM_FORMATADA|
+---------+-----------------+---------------+---------+-----------------+---------------------+------------------+
|  Negocio|      Fort Pierce|    Fort Pierce|       51|      Alimentação|           2016-01-01|        2016-01-01|
|  Negocio|      Fort Pierce|    Fort Pierce|        5|             null|           2016-01-02|        2016-01-02|
|  Negocio|      Fort Pierce|    Fort Pierce|       48|         Entregas|           2016-01-02|        2016-01-02|
|  Negocio|      Fort Pierce|    Fort Pierce|       47|          Reunião|           2016-01-05|        2016-01-05|
|  Negocio|      Fort Pierce|West Palm Beach|      637|Visita ao cliente|           2016-01-06|        2016-01-06|
|  Negocio|  West Palm Beach|West Palm Beach|       43|      Alimentação|       

None

In [ ]:
info_transportes.printSchema()

root
 |-- CATEGORIA: string (nullable = true)
 |-- LOCAL_INICIO: string (nullable = true)
 |-- LOCAL_FIM: string (nullable = true)
 |-- DISTANCIA: integer (nullable = true)
 |-- PROPOSITO: string (nullable = true)
 |-- DATA_INICIO_FORMATADA: string (nullable = true)
 |-- DATA_FIM_FORMATADA: string (nullable = true)



In [ ]:
# Renomeando as colunas com as nova formatacao
info_transportes = info_transportes.withColumnRenamed('DATA_INICIO_FORMATADA', 'DATA_INICIO')
info_transportes = info_transportes.withColumnRenamed('DATA_FIM_FORMATADA', 'DATA_FIM')

In [ ]:
display(
    info_transportes.show()
)

+---------+-----------------+---------------+---------+-----------------+-----------+----------+
|CATEGORIA|     LOCAL_INICIO|      LOCAL_FIM|DISTANCIA|        PROPOSITO|DATA_INICIO|  DATA_FIM|
+---------+-----------------+---------------+---------+-----------------+-----------+----------+
|  Negocio|      Fort Pierce|    Fort Pierce|       51|      Alimentação| 2016-01-01|2016-01-01|
|  Negocio|      Fort Pierce|    Fort Pierce|        5|             null| 2016-01-02|2016-01-02|
|  Negocio|      Fort Pierce|    Fort Pierce|       48|         Entregas| 2016-01-02|2016-01-02|
|  Negocio|      Fort Pierce|    Fort Pierce|       47|          Reunião| 2016-01-05|2016-01-05|
|  Negocio|      Fort Pierce|West Palm Beach|      637|Visita ao cliente| 2016-01-06|2016-01-06|
|  Negocio|  West Palm Beach|West Palm Beach|       43|      Alimentação| 2016-01-06|2016-01-06|
|  Negocio|  West Palm Beach|     Palm Beach|       71|          Reunião| 2016-01-06|2016-01-06|
|  Negocio|             Cary| 

None

In [ ]:
from pyspark.sql.functions import *

# Criando novo dataframe info_corridas_do_dia

info_corridas_do_dia = info_transportes.groupBy("DATA_INICIO").agg(
    count("*").alias("QT_CORR"),
    count(when(col("CATEGORIA") == "Negocio", True)).alias("QT_CORR_NEG"),
    count(when(col("CATEGORIA") == "Pessoal", True)).alias("QT_CORR_PESS"),
    max("DISTANCIA").alias("VL_MAX_DIST"),
    min("DISTANCIA").alias("VL_MIN_DIST"),
    round(avg("DISTANCIA"), 2).alias("VL_AVG_DIST"),
    count(when(col("PROPOSITO") == "Reunião", True)).alias("QT_CORR_REUNI"),
    count(when((col("PROPOSITO") != "Reunião") & col("PROPOSITO").isNotNull(), True)).alias("QT_CORR_NAO_REUNI")
).withColumnRenamed("DATA_INICIO", "DATA_REFE")

In [ ]:
info_corridas_do_dia.show()


+----------+-------+-----------+------------+-----------+-----------+-----------+-------------+-----------------+
| DATA_REFE|QT_CORR|QT_CORR_NEG|QT_CORR_PESS|VL_MAX_DIST|VL_MIN_DIST|VL_AVG_DIST|QT_CORR_REUNI|QT_CORR_NAO_REUNI|
+----------+-------+-----------+------------+-----------+-----------+-----------+-------------+-----------------+
|2016-08-17|      8|          8|           0|        121|         14|      60.38|            0|                1|
|2016-04-22|      4|          4|           0|        225|         10|     118.75|            4|                0|
|2016-08-08|      5|          5|           0|         52|          4|       32.6|            0|                2|
|2016-09-11|      2|          2|           0|         98|         86|       92.0|            0|                0|
|2016-07-06|      5|          5|           0|         99|         31|       65.0|            1|                3|
|2016-03-17|      7|          3|           4|        302|         17|      127.0|       